# Organize data for running PCA

Two main approaches:
1. select a subset of all ontogeny sessions, a few sessions from each age and sex
2. select a subset of all longtogeny sessions, a few sessions from each age and sex. Make sure to have the same number of sessions and similar ages per mouse.

In [36]:
import h5py
import datetime
import pandas as pd
from tqdm.auto import tqdm
from pathlib import Path
from toolz import concat, curry

In [3]:
session_exclusion_list = '/n/groups/datta/win/longtogeny/data/metadata/Ontogeny experiment list - Session quality control.csv'
exclusion_df = pd.read_csv(session_exclusion_list, header=0)

In [5]:
exclusion_df['Keep?'] = exclusion_df['Keep?'].str.lower()

In [6]:
exclusion_df['Keep?'].unique()

array(['y', 'n', '?'], dtype=object)

In [13]:
exclusions = exclusion_df.query('`Keep?` != "y"')

In [20]:
exclusions.head()

,File path,Checked?,Keep?,Issue
3,/n/groups/datta/Dana/Ontogeny/raw_data/Ontogen...,y,n,"fading, noise, further inspection needed"
4,/n/groups/datta/Dana/Ontogeny/raw_data/Ontogen...,y,n,noise
5,/n/groups/datta/Dana/Ontogeny/raw_data/Ontogen...,y,n,wall noise
6,/n/groups/datta/Dana/Ontogeny/raw_data/Ontogen...,y,n,replaced with noise
7,/n/groups/datta/Dana/Ontogeny/raw_data/Ontogen...,y,n,wall noise


In [21]:
def is_size_normalized(file):
    with h5py.File(file, 'r') as h5f:
        return 'win_size_norm_frames' in h5f


def not_excluded(file):
    return file not in exclusions['File path'].values

## Organize ontogeny sessions

These are the data that will be run through PCA and the modeling steps

- create a dataframe with the file path, age, sex, and session name
- do a groupby and subsample the same number of sessions for each (age, sex) group
- symlink results.h5 files to a new folder

In [40]:
agg_folder = Path('/n/groups/datta/win/longtogeny/data/ontogeny/version_02/training_data')
agg_folder.mkdir(exist_ok=True, parents=True)

In [41]:
folders = [
    '/n/groups/datta/Dana/Ontogeny/raw_data/Ontogeny_females',
    '/n/groups/datta/Dana/Ontogeny/raw_data/Ontogeny_males',
]

folders = [Path(f) for f in folders]

In [42]:
@curry
def thread_and(funcs, x):
    return all(f(x) for f in funcs)

In [43]:
# filter files for the ones with size_normalized_frames
files = sorted(
    filter(
        thread_and((is_size_normalized, not_excluded)),
        concat(f.glob("**/results_00.h5") for f in folders),
    )
)

In [45]:
df = []
for f in files:
    data = dict(
        age=f.parents[2].name,
        session=f.parents[1].name,
        sex=f.parents[3].name.split('_')[-1][:-1],
        path=str(f)
    )
    df.append(data)
df = pd.DataFrame(df)

In [46]:
df.groupby(['age', 'sex']).count()

,,session,path
age,sex,,
12months,female,24,24
12months_28042021,male,26,26
18months,female,24,24
18months_29042021,male,27,27
22months_06052023,male,32,32
3months,female,24,24
3months_19042021,male,27,27
3wks,female,24,24
3wks_02112021,male,16,16


In [47]:
# sub-select 3 sessions from each group
# 51 sessions = 1.8e6 frames
sample = df.groupby(['age', 'sex']).sample(n=3, random_state=0)

In [48]:
for path in map(Path, sample['path']):
    try:
        new_path = agg_folder / (path.parents[1].name + '.h5')
        new_path.symlink_to(path)
        new_path = agg_folder / (path.parents[1].name + '.yaml')
        new_path.symlink_to(path.with_suffix('.yaml'))
    except FileExistsError:
        print(path, 'exists in training set')

## Organize longtogeny sessions

In [49]:
agg_folder = Path('/n/groups/datta/win/longtogeny/data/longtogeny/version_02/training_data')
agg_folder.mkdir(exist_ok=True, parents=True)

In [50]:
folders = [
    '/n/groups/datta/Dana/Ontogeny/raw_data/longtogeny_pre_unet/Females',
    '/n/groups/datta/Dana/Ontogeny/raw_data/longtogeny_pre_unet/Males',
]
folders = [Path(f) for f in folders]

In [51]:
# filter files for the ones with size_normalized_frames
files = sorted(
    filter(
        thread_and((is_size_normalized, not_excluded)),
        concat(f.glob("**/results_00.h5") for f in folders),
    )
)

In [52]:
len(files)

1651

In [53]:
with h5py.File(files[0], 'r') as h5f:
    print(h5f['metadata/acquisition/SessionName'][()].decode())
    print(h5f['metadata/acquisition/SubjectName'][()].decode())

002
01_01_001


In [54]:
df = []
for f in tqdm(files):
    try:
        with h5py.File(f, 'r') as h5f:
            session = h5f['metadata/acquisition/SubjectName'][()].decode()
            start = h5f['metadata/acquisition/StartTime'][()].decode()
        sex = f.parents[3].name.lower()[:-1]
        sex = 'female' if 'Females' in str(f) else 'male'
        data = dict(
            mouse=session[:5],
            subject_name=session,
            session=f.parents[1].name,
            sex=sex,
            date=datetime.datetime.strptime(start.split('.')[0], '%Y-%m-%dT%H:%M:%S'),
            path=str(f)
        )
        df.append(data)
    except KeyError:
        print(f, 'incomplete')
df = pd.DataFrame(df)

  0%|          | 0/1651 [00:00<?, ?it/s]

/n/groups/datta/Dana/Ontogeny/raw_data/longtogeny_pre_unet/Males/20210403/session_20210403155244/proc/results_00.h5 incomplete
/n/groups/datta/Dana/Ontogeny/raw_data/longtogeny_pre_unet/Males/20210406/session_20210406131627/proc/results_00.h5 incomplete
/n/groups/datta/Dana/Ontogeny/raw_data/longtogeny_pre_unet/Males/20210410/session_20210410184436/proc/results_00.h5 incomplete
/n/groups/datta/Dana/Ontogeny/raw_data/longtogeny_pre_unet/Males/20210410/session_20210410205337/proc/results_00.h5 incomplete
/n/groups/datta/Dana/Ontogeny/raw_data/longtogeny_pre_unet/Males/20210423/session_20210423193537/proc/results_00.h5 incomplete
/n/groups/datta/Dana/Ontogeny/raw_data/longtogeny_pre_unet/Males/20210514/session_20210514153543/proc/results_00.h5 incomplete
/n/groups/datta/Dana/Ontogeny/raw_data/longtogeny_pre_unet/Males/20210521/session_20210521154512/proc/results_00.h5 incomplete
/n/groups/datta/Dana/Ontogeny/raw_data/longtogeny_pre_unet/Males/20210526/session_20210526150505/proc/results_0

In [60]:
filt_df = df[~df['mouse'].str.lower().str.contains('c') & ~df['mouse'].str.lower().str.contains('long')]

In [61]:
filt_df.groupby('mouse').count()

,subject_name,session,sex,date,path
mouse,,,,,
01_01,94,94,94,94,94
01_02,93,93,93,93,93
01_03,96,96,96,96,96
01_04,92,92,92,92,92
02_01,98,98,98,98,98
02_02,94,94,94,94,94
02_03,91,91,91,91,91
02_04,77,77,77,77,77
03_01,94,94,94,94,94


In [62]:
sample = filt_df.groupby('mouse').sample(n=3, random_state=0)

In [63]:
for path in map(Path, sample['path']):
    try:
        new_path = agg_folder / (path.parents[1].name + '.h5')
        new_path.symlink_to(path)
        new_path = agg_folder / (path.parents[1].name + '.yaml')
        new_path.symlink_to(path.with_suffix('.yaml'))
    except FileExistsError:
        print(path, 'exists in training set')